In [51]:
import pandas as pd

# Check if this works
def set_pd_options(rows=1000,cols=30):
    import pandas as pd
    try:
        pd.set_option('display.max_rows', rows)
        pd.set_option('display.max_columns', cols)
        return True
    except:
        print("Error on input, must be an integer")
        return False
# read excel given filename
def read_excel_to_df(file='GSAF5.xls'):
    import pandas as pd
    try:
        df = pd.read_excel(file)
        return df
    except:
        print("Cannot read file, please make sure you added the correct extension or file path")
        return None
# rename columns removing spaces and putting to lower   
def rename_columns(df):
    # rename columns
    df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()
    return df
# helper function for clear_country. Given a string, if it eds with '?' remove it at the end (apply)    
def clean_char(el,char='?'):
    if el.endswith(char):
        return el[:-1]
    else:
        return el    
# clean country varible
def clean_country(df):
    df['country'] = df['country'].fillna('').str.lower().str.strip() # fill na with '', put lowercase and remove spaces
    df['country'] = df['country'].apply(clean_char)  # to lower, remove ? at the end
    df['country'] = df['country'].str.replace(' (uae)','').str.replace('&','and').str.replace(' (sri lanka)','').str.replace('columbia','colombia').str.replace('coast of africa','africa').str.replace('st. martin','st martin').str.replace('st. maartin','')
    return df
# cleaner for activity
def clean_helper(row,mydict):
    for key,value in mydict.items():
        if key in row.lower():
            return value
    return row

def clean_activity(df):
    mydict = {'diving':'diving',
          'accident':'accident',
          'waves':'surfing',
          'adrift':'adrift',
          'attempting':'attempt',
          'attempted':'attempt',
          'bathing':'bathing',
          'boat':'boat',
          'body':'boogie',
          'boogie':'boogie',
           'canoe':'canoe',
          'collecting':'collect',
          'dived':'diving',
          'diving': 'diving',
          'fishing':'fishing',
          'swimming':'swimming',
          'disaster':'sea disaster',
          'boat':'sea disaster',
          'air':'sea disaster',
          'ship':'sea disaster',
          'typhoon':'sea disaster',
          'cyclone':'sea disaster',
          'beach':'beach',
          'feeding':'feeding',
          'fell':'fell',
          'floating':'floating',
          'jumped':'jumped',
          'jumping':'jumped',
          'kayak':'kayak',
          'kite':'kite',
          'paddle':'paddling',
          'paddling':'paddling',
          'surf':'surfing',
          'sailing':'sea disaster',
          'sea disaster':'sea disaster',
          'sinking':'sinking',
          'snorkeling':'snorkeling',
          'spearfishing':'fishing',
          'standing':'standing',
          'treading': 'swimming',
          'photo': 'photography',
          'wading': 'swimming',
          'wade':'swimming',
          'washing':'washing',
          'wreck':'sea disaster',
          'yacht':'sea disaster'
          }
    df['activity'] = df['activity'].fillna('not_confirmed').str.lower().str.strip()
    df['activity'] = df['activity'].apply(clean_helper,mydict=mydict)
    return df

def clean_type(df):
    df["type"] = df["type"].fillna('Unconfirmed')
    df["type"] = df["type"].replace({"?": "Unconfirmed", "Unconfirmed": "Unconfirmed", "Unverified": "Unconfirmed", "Under investigation":"Unconfirmed","Questionable":"Unconfirmed"})
    df["type"] = df["type"].str.lower()
    return df

# main
# set options and start df cleaning
set_pd_options()
df = read_excel_to_df('GSAF5.xls')
df = rename_columns(df)
df = df.drop(['unnamed:_21','unnamed:_22'],axis=1) # remove last two columns (NaN mostly in all, not relevant

# clean variables
df = clean_country(df)
df = clean_activity(df)
df = clean_type(df)
# Needs fixing / observations:
# Sex is missing in # cells (nan) nad has 8 cells
# There are names replaces by "male" and "Female"
# last two columns can be removed, (NaN as value)
# columns after 'source' were hidden, althoguh does not seem to relevant
# Going to work with Activity, Country, Type. Clean only these values

# Hipothesis
"""Null Hypothesis (H0): The number of shark attacks during surfing is not significantly higher in the USA compared to other countries, and there is no association between the popularity of surfing and shark attacks.

Alternative Hypothesis (H1): Most shark attacks occur during surfing, with a higher frequency in the USA due to the popularity of surfing as a sport, and these attacks are unprovoked."""
# most attacks are done when surfing and are located in the USA since surfing is a more popular sport there and it as unprovoked (explore more, see what other data is relevant and why)
# There are more attacks on America than other continent since they do more beach activities (Second hipothesis)


print("activity after cleaning\n", df['activity'].value_counts())
print("country after cleaning\n", df['country'].value_counts())



pivot_table = df.pivot_table(index='country',columns='activity',values='date', aggfunc='count')


# Check hipothesis country, type against activity
pivot_table2 = df.pivot_table(index=['country','type'], 
                             columns='activity',values='date', aggfunc='count').fillna(0)



top_3_countries = pivot_table.sum(axis=1).sort_values(ascending=False).head(3).index
top_3_activity = pivot_table.sum(axis=0).sort_values(ascending=False).head(3).index
print(pivot_table.loc[top_3_countries,top_3_activity])
print(pivot_table2.loc[top_3_countries,top_3_activity])


activity
swimming    1387
Name: count, dtype: int64


activity                   swimming  surfing  fishing
country      type                                    
usa          boat               0.0      0.0      1.0
             invalid           44.0     31.0     16.0
             provoked          13.0     12.0    124.0
             sea disaster       1.0      0.0      1.0
             unconfirmed        3.0      3.0      2.0
             unprovoked       493.0    675.0    135.0
             watercraft         0.0      0.0     38.0
australia    boat               0.0      0.0      1.0
             invalid           28.0     10.0     17.0
             provoked           5.0      3.0     74.0
             sea disaster       2.0      0.0      2.0
             unconfirmed        1.0      4.0      3.0
             unprovoked       216.0    252.0    115.0
             watercraft         0.0      1.0     67.0
south africa boat               0.0      0.0      1.0
             invalid           14.0      3.0     11.0
             provoked           0.0      0.0     34.0
             sea disaster       0.0      0.0      1.0
             unconfirmed        0.0      0.0      0.0
             unprovoked       120.0    120.0     51.0
             watercraft         0.0      1.0     34.0

In [14]:

pt_general_activity = df.pivot_table(index='country', values='activity', aggfunc='count')
pt_general_activity = pt_general_activity.sort_values(by='activity', ascending=False)
#df.info()
sorted(df['injury'].value_counts().index)

filter_fatal = (df['injury'])


[' ',
 ' "Struck across loins" but no injury. According to witnesses, incident involved a bottlenose dolphin.',
 ' 2 of the 5 fishermen were so seriously injured they died of their wounds',
 ' 3 fingers & thigh lacerated, foot crushed',
 ' 3 unidentified bodies recovered (male, female & young girl) from gut of female  4.5 m [14\'9"]  white shark caught in fishing net. They may have drowned in tidal wave following earthquake ',
 ' 55  perished, some were  taken by sharks',
 ' 6 lacerations to left hand',
 ' Ankle lacerated',
 ' Dinghy damaged by shark. No injurty to the 5 men onboard',
 " Du Val's leg was bitten but he survived",
 ' FATAL',
 ' FATAL. Shark bite was minor injury, but he suffered a heart attack afterwards and died 6 hours later',
 ' Forced at gunpoint to jump overboard. Presumed fatal; shark involvement probable but not confirmed ',
 ' His gold watch was later found in a shark but death may have been due to drowning',
 ' No injury',
 ' No injury. Shark bumped kayak, fling

In [ ]:


df['country'] = df['country'].fillna('').str.lower().str.strip() # fill na with '', put lowercase and remove spaces
df['country'] = df['country'].apply(clean_char)  # to lower, remove ? at the end
df['country'] = df['country'].str.replace(' (uae)','').str.replace('&','and').str.replace(' (sri lanka)','').str.replace('columbia','colombia').str.replace('coast of africa','africa').str.replace('st. martin','st martin').str.replace('st. maartin','')

#Why it does not work?
#df['country'] = df['country'].map({'st. martin': 'st martin','st. maartin':'st martin'},na_action='ignore')
sorted(df['country'].unique())
# will need / to be removed if we want to plot countries. How to copy the row in that case?
# diego garcia?

In [ ]:
mydict = {'diving':'diving',
          'accident':'accident',
          'waves':'surfing',
          'adrift':'adrift',
          'attempting':'attempt',
          'attempted':'attempt',
          'bathing':'bathing',
          'boat':'boat',
          'body':'boogie',
          'boogie':'boogie',
           'canoe':'canoe',
          'collecting':'collect',
          'dived':'diving',
          'diving': 'diving',
          'fishing':'fishing',
          'swimming':'swimming',
          'disaster':'sea disaster',
          'boat':'sea disaster',
          'air':'sea disaster',
          'ship':'sea disaster',
          'typhoon':'sea disaster',
          'cyclone':'sea disaster',
          'beach':'beach',
          'feeding':'feeding',
          'fell':'fell',
          'floating':'floating',
          'jumped':'jumped',
          'jumping':'jumped',
          'kayak':'kayak',
          'kite':'kite',
          'paddle':'paddling',
          'paddling':'paddling',
          'surf':'surfing',
          'sailing':'sea disaster',
          'sea disaster':'sea disaster',
          'sinking':'sinking',
          'snorkeling':'snorkeling',
          'spearfishing':'fishing',
          'standing':'standing',
          'treading': 'swimming',
          'photo': 'photography',
          'wading': 'swimming',
          'wade':'swimming',
          'washing':'washing',
          
          }

def clean_helper(row,mydict):
    for key,value in mydict.items():
        if key in row.lower():
            return value
    return row

def clean_activity(df):
    mydict = {'diving':'diving',
          'accident':'accident',
          'waves':'surfing',
          'adrift':'adrift',
          'attempting':'attempt',
          'attempted':'attempt',
          'bathing':'bathing',
          'boat':'boat',
          'body':'boogie',
          'boogie':'boogie',
           'canoe':'canoe',
          'collecting':'collect',
          'dived':'diving',
          'diving': 'diving',
          'fishing':'fishing',
          'swimming':'swimming',
          'disaster':'sea disaster',
          'boat':'sea disaster',
          'air':'sea disaster',
          'ship':'sea disaster',
          'typhoon':'sea disaster',
          'cyclone':'sea disaster',
          'beach':'beach',
          'feeding':'feeding',
          'fell':'fell',
          'floating':'floating',
          'jumped':'jumped',
          'jumping':'jumped',
          'kayak':'kayak',
          'kite':'kite',
          'paddle':'paddling',
          'paddling':'paddling',
          'surf':'surfing',
          'sailing':'sea disaster',
          'sea disaster':'sea disaster',
          'sinking':'sinking',
          'snorkeling':'snorkeling',
          'spearfishing':'fishing',
          'standing':'standing',
          'treading': 'swimming',
          'photo': 'photography',
          'wading': 'swimming',
          'wade':'swimming',
          'washing':'washing',
          }
    df['activity'] = df['activity'].fillna('not_confirmed').str.lower().str.strip()
    df['activity'] = df['activity'].apply(clean_helper,mydict=mydict)
    return df
    
activity_values = df['activity'].fillna('not_confirmed').unique() # We dot know the activity, so use not_confirmed to fill 
sorted(activity_values)

df['activity'] = df['activity'].fillna('not_confirmed').str.lower().str.strip()
print("len before apply:", df['activity'].value_counts())

df['activity'] = df['activity'].apply(clean_helper,mydict=mydict)   # 581 not filled..
print("after apply:", df['activity'].value_counts())

sorted(df['activity'].value_counts().index)

# might try injury column
# swimming and fishing
# 

In [ ]:
filter_surf = df['activity'] == 'surfing'
x = df.loc[filter_surf]
x['activity'].value_counts()

In [ ]:
type_values = df['type'].unique() # limpiar Jose
type_values

# replace nan, ?, Unconfirmed, Invalid with Unverified. Use map.
# José 11:45pm

""" 
I remove the nan variable,replace some variables like ? to Unconfirmed,
and turn all the variables in lower case
"""
# replace nan, ?, Unconfirmed, Questionable with Unverified. Used replace (Jose). Invalid, assume is not an attack
df["type"] = df["type"].fillna('Unconfirmed')
df["type"] = df["type"].replace({"?": "Unconfirmed", "Unconfirmed": "Unconfirmed", "Unverified": "Unconfirmed", "Under investigation":"Unconfirmed","Questionable":"Unconfirmed"})
df["type"] = df["type"].str.lower()
df["type"].unique()
df["type"].value_counts()

In [ ]:
pd.set_option('display.max_rows', 1000)

pivot_table = df.pivot_table(index='country', 
                             columns='activity',values='date', aggfunc='count')
#sorted_pivot_table = pivot_table.sort_values(by='activity', ascending=False)

#surfing_stats = sorted_pivot_table[sorted_pivot_table.index.isin(['surfing'])]

pivot_table.loc[['usa','australia','south africa'],['surfing','swimming']]


In [ ]:
"""type
unprovoked      5089
provoked         633
invalid          552
watercraft       355
sea disaster     242
questionable      20
boat               7
unconfirmed        4
Name: count, dtype: int64 """